# Log Creation and Testing

Suppose we have created our chart and we calculated the MACD and RSI. 

In [87]:
import math
import numpy
import random
import decimal
import scipy.linalg
import matplotlib
import numpy.random as nrand
import matplotlib.pyplot as plt
import pandas
from pandas import DataFrame

InitialPrice = 1000
RandomInc = numpy.empty(15000)
RandomPrice = numpy.empty(15000)
decision = numpy.zeros(len(RandomPrice))
RandomPrice[0] = InitialPrice
    
for i in range(15000):
    suddenjump = random.uniform(0,1)
    if suddenjump < 0.1:
        RandomInc[i] = random.uniform(-0.05,0.05)
    else:
        RandomInc[i] = random.uniform(-0.01,0.01)
            
    if i!=14999:
        if RandomPrice[i]+RandomPrice[i]*RandomInc[i] > 50:
            RandomPrice[i+1] = RandomPrice[i]+RandomPrice[i]*RandomInc[i]
        else:   
            RandomPrice[i+1] = RandomPrice[i]-RandomPrice[i]*RandomInc[i]

df = DataFrame(RandomPrice,columns=['Prices'])
df['Perc'] = RandomInc
df.insert(2,'Decision',decision)

exp1 = df['Prices'].ewm(span=12, adjust=False).mean()
exp2 = df['Prices'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
exp3 = macd.ewm(span=9, adjust=False).mean()
df['MACD']=macd
df['Signal Line']=exp3
df['MACD'][0:17]=0
df['Signal Line'][0:26]=0

delta = df['Prices'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
    
df['RSI'] = 100 - (100/(1+rs))
df['RSI'][0:14]=0
print(df[60:65])

        Prices      Perc  Decision      MACD  Signal Line        RSI
60  928.210538 -0.007118       0.0 -3.318419    -3.145233  45.180065
61  921.603817 -0.007030       0.0 -3.480796    -3.212345  41.512129
62  915.124967  0.003810       0.0 -4.085179    -3.386912  38.234034
63  918.611554  0.005111       0.0 -4.234012    -3.556332  40.937052
64  923.306927  0.008110       0.0 -3.927808    -3.630627  44.461935


Now we'll write our decision mechanism. 

If RSI>70 and MACD Line is smaller than Signal Line this is an indicator for our mechanism to long the stock.
If RSI<30 and MACD Line is bigger than Signal Line this is an indicator for our mechanism to short the stock.

This decision mechanism is very basic and not trustable at all. 
However, since this project is a start to creating a more complex system it is fine. Our goal is to be able to create a system that creates logs and test how the decision mechanism works. Thus, creating a decision mechanism that is working well will be a problem for the future.

Now let's add the decisions to the dataframe.

In [88]:
for i in range(len(df)):
    if df.iloc[i]['RSI']>70 and df.iloc[i]['MACD']<df.iloc[i]['Signal Line']:
        df.loc[i]['Decision']=-1
    elif df.iloc[i]['RSI']<30 and df.iloc[i]['MACD']>df.iloc[i]['Signal Line']:
        df.loc[i]['Decision']=1
    else:
        df.loc[i]['Decision']=0
        
print(df[60:65])

        Prices      Perc  Decision      MACD  Signal Line        RSI
60  928.210538 -0.007118       0.0 -3.318419    -3.145233  45.180065
61  921.603817 -0.007030       0.0 -3.480796    -3.212345  41.512129
62  915.124967  0.003810       0.0 -4.085179    -3.386912  38.234034
63  918.611554  0.005111       0.0 -4.234012    -3.556332  40.937052
64  923.306927  0.008110       0.0 -3.927808    -3.630627  44.461935


In this part, we are going to create logs.
In our log, we'll keep;
- the Start Capital before the the long or short
- The Position Taken
- The StartPrice when we opened the position
- How Many of the Stock is bought
- At what time we opened the position
- When we closed the position
- The duration of the position
- The End Price when we closed the position
- The End Capital we gathered
- The +/-
- And the Perc on our Capital

In [89]:
position=0
startpos=0

initialmoney=1000

log = DataFrame(columns=['StartCapital','Position Taken','startprice','HowMany','StartTime','EndTime','Duration','endprice','EndCapital','ProfitLoss','Perc'],index=range(1000))
logcounter = 0

In [90]:
for i in range(1,len(df)):
    if position!=0 and i!=len(df)-1:
            
        if df.iloc[i-1]['Decision']!=0:
            if logcounter==0:
                log.loc[logcounter]['StartCapital']=initialmoney
                
            log.loc[logcounter]['Position Taken']=position
            log.loc[logcounter]['startprice']=(df.iloc[i]['Prices']-df.iloc[i-1]['Prices'])*random.uniform(-1,1)+df.iloc[i-1]['Prices']
            log.loc[logcounter]['HowMany']=(log.loc[logcounter]['StartCapital']/df.iloc[i]['Prices'])*0.999
            log.loc[logcounter]['StartTime']=i
            startpos=i

        if position==1:
            if df.loc[i]['Decision']==-1 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=-1
            elif df.loc[i]['Decision']==1 and i-startpos+1>3:
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=1
            elif df.iloc[i]['Prices']>=log.iloc[logcounter]['startprice']*1.2 or df.iloc[i]['Prices']<=log.iloc[logcounter]['startprice']*0.8: 
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=0
                    
        elif position==-1:
            if df.loc[i]['Decision']==1 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=1
            elif df.loc[i]['Decision']==-1 and i-startpos+1>3 :
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1
                position=-1
            elif df.iloc[i]['Prices']<=log.iloc[logcounter]['startprice']*0.8 or df.iloc[i]['Prices']>=log.iloc[logcounter]['startprice']*1.2:
                log.loc[logcounter]['EndTime']=i
                log.loc[logcounter]['Duration']=i-startpos+1
                log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
                log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
                log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
                log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
                log.iloc[logcounter+1]['StartCapital']=log.iloc[logcounter]['EndCapital']
                logcounter+=1 
                position=0       
                    
    elif i==len(df)-1:
        if position==1:
            log.loc[logcounter]['EndTime']=i
            log.loc[logcounter]['Duration']=i-startpos+1
            log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
            log.loc[logcounter]['EndCapital']=log.iloc[logcounter]['HowMany']*log.iloc[logcounter]['endprice']*0.999
            log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
            log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']
            position=0
        elif position==-1:
            log.loc[logcounter]['EndTime']=i
            log.loc[logcounter]['Duration']=i-startpos+1
            log.loc[logcounter]['endprice']=df.iloc[i]['Prices']
            log.loc[logcounter]['EndCapital']=(log.iloc[logcounter]['StartCapital']+log.iloc[logcounter]['HowMany']*(log.iloc[logcounter]['startprice']-log.loc[logcounter]['endprice']))*0.999
            log.loc[logcounter]['ProfitLoss']=log.iloc[logcounter]['EndCapital']-log.iloc[logcounter]['StartCapital']
            log.loc[logcounter]['Perc']=log.loc[logcounter]['ProfitLoss']/log.iloc[logcounter]['StartCapital']

            position=0
    else:
        if df.loc[i]['Decision']==1:
            position=1
        elif df.loc[i]['Decision']==-1:
            position=-1
        else:
            position=0
            

log=log.dropna()
print(log)

   StartCapital Position Taken startprice   HowMany StartTime EndTime  \
0          1000              1    929.478   1.05776        31     296   
1       770.729              1    679.926   1.12398       339     693   
2       609.658              1    526.354   1.15882       710     850   
3       732.256              1    572.712    1.2606       915     928   
4       700.454              1    558.999    1.2652       929     950   
5       678.965              1    538.175   1.26034       951    1264   
6       731.684             -1    581.757   1.26703      1265    1296   
7       637.848             -1    660.007  0.981911      1297    1303   
8       635.105             -1    668.968  0.923673      1305    1415   
9       671.987              1    625.496   1.07067      1418    1421   
10      666.434              1     617.37   1.07199      1424    1768   
11      529.469              1    495.124   1.05015      1769    1773   
12       512.12              1    486.713   1.09735

See the decisions:

In [91]:
countlp=0
countln=0
countsp=0
countsn=0

for i in range(len(log)):
    if log.iloc[i]['Position Taken']==1 and log.iloc[i]['ProfitLoss']>0:
        countlp += 1
    elif log.iloc[i]['Position Taken']==1 and log.iloc[i]['ProfitLoss']<0:
        countln += 1
    elif log.iloc[i]['Position Taken']==-1 and log.iloc[i]['ProfitLoss']>0: 
        countsp += 1
    elif log.iloc[i]['Position Taken']==-1 and log.iloc[i]['ProfitLoss']<0:
        countsn += 1

df2=DataFrame(columns=['Total Count'],index=[-1,1])

df2['Total Count']=[countsp+countsn,countlp+countln]
df2['Correct Decision']=[countsp,countlp]
df2['Wrong Decision']=[countsn,countln]
df2['Accuracy']=[countlp/(countlp+countln),countsp/(countsp+countsn)]
df2.loc['Total']=[sum(df2['Total Count']),countlp+countsp,countln+countsn,(countlp+countsp)/(countlp+countsp+countln+countsn)]

print(df2)

       Total Count  Correct Decision  Wrong Decision  Accuracy
-1            27.0              12.0            15.0  0.280000
1             50.0              14.0            36.0  0.444444
Total         77.0              26.0            51.0  0.337662


Log Profit or Loss:

In [92]:
totallog = DataFrame(columns=['Initial Money','Profit or Loss','Perc'],index=range(1))

In [93]:
totallog['Initial Money']=initialmoney
totallog['Profit or Loss']=log.iloc[-1]['EndCapital']-initialmoney
totallog['Perc']=(log.iloc[-1]['EndCapital']-initialmoney)*100/initialmoney
print(totallog)

   Initial Money  Profit or Loss       Perc
0           1000     -435.222082 -43.522208
